In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
mohamedmustafa_real_life_violence_situations_dataset_path = kagglehub.dataset_download('mohamedmustafa/real-life-violence-situations-dataset')

print('Data source import complete.')


100%|██████████| 3.58G/3.58G [00:16<00:00, 239MB/s]

Extracting files...


Data source import complete.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
     for filename in filenames:
         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Library

In [4]:
import os
import shutil
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow
import keras
from collections import deque
from collections import deque
import seaborn as sns
sns.set_theme()  # Thiết lập theme mặc định của Seaborn

%matplotlib inline

from sklearn.model_selection import train_test_split

from keras.layers import *
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

# Visualize the Data

In [5]:
from IPython.display import HTML
from base64 import b64encode

# To Show a Video in Notebook
def Play_Video(filepath):
    html = ''
    video = open(filepath,'rb').read()
    src = 'data:video/mp4;base64,' + b64encode(video).decode()
    html += '<video width=640 muted controls autoplay loop><source src="%s" type="video/mp4"></video>' % src
    return HTML(html)

Extracting Frames

In [6]:
# Directories for video classification
non_violence_videos_path = "/root/.cache/kagglehub/datasets/mohamedmustafa/real-life-violence-situations-dataset/versions/1/Real Life Violence Dataset/NonViolence"
violence_videos_path = "/root/.cache/kagglehub/datasets/mohamedmustafa/real-life-violence-situations-dataset/versions/1/Real Life Violence Dataset/Violence"

# Get the list of video files from each directory
non_violence_video_list = os.listdir(non_violence_videos_path)
violence_video_list = os.listdir(violence_videos_path)

# Select a random video file from each category
random_non_violence_video = random.choice(non_violence_video_list)
random_violence_video = random.choice(violence_video_list)

In [7]:
# Play a non-violence video
display(Play_Video(f"{non_violence_videos_path}/{random_non_violence_video}"))


In [8]:
# Play a violence video
display(Play_Video(f"{violence_videos_path}/{random_violence_video}"))

In [10]:
# Define constants for video frame dimensions and sequence length
frame_height, frame_width = 96, 96
sequence_length = 16

# Path to the dataset
dataset_path = "/root/.cache/kagglehub/datasets/mohamedmustafa/real-life-violence-situations-dataset/versions/1/Real Life Violence Dataset"

# Class labels for classification
class_labels = ["NonViolence", "Violence"]

print(frame_height)

96


In [11]:
def extract_frames(video_path):
    """Extract a fixed number of frames from a video file."""
    frames = []

    # Open the video file
    video_capture = cv2.VideoCapture(video_path)
    if not video_capture.isOpened():
        raise ValueError(f"Error opening video file: {video_path}")

    # Get the total number of frames in the video
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames == 0:
        video_capture.release()
        raise ValueError(f"Video has no frames: {video_path}")

    # Calculate the interval to skip frames
    frame_interval = max(total_frames // sequence_length, 1)

    # Extract frames at regular intervals
    for frame_index in range(sequence_length):
        video_capture.set(cv2.CAP_PROP_POS_FRAMES, frame_index * frame_interval)
        success, frame = video_capture.read()
        if not success:
            print(f"Warning: Could not read frame {frame_index} from {video_path}")
            break

        resized_frame = cv2.resize(frame, (frame_width, frame_height))
        normalized_frame = resized_frame.astype('float32') / 255.0
        frames.append(normalized_frame)

    video_capture.release()

    if len(frames) != sequence_length:
        print(f"Warning: Extracted frames ({len(frames)}) less than required ({sequence_length}) for {video_path}")

    return frames

# Build Dataset

In [12]:
def build_dataset(dataset_path, class_labels, sequence_length, frame_extraction_func):
    """Create a dataset by extracting frames from videos and assigning labels."""
    features, labels, video_paths = [], [], []

    for class_index, class_name in enumerate(class_labels):
        print(f"Processing class: {class_name} ({class_index + 1}/{len(class_labels)})")

        class_dir = os.path.join(dataset_path, class_name)
        video_files = [f for f in os.listdir(class_dir) if f.endswith('.mp4') or f.endswith('.avi')]

        for video_file in video_files:
            video_path = os.path.join(class_dir, video_file)
            try:
                frames = frame_extraction_func(video_path)
                if len(frames) == sequence_length:
                    features.append(frames)
                    labels.append(class_index)
                    video_paths.append(video_path)
                else:
                    print(f"Skipping {video_file}: Not enough frames ({len(frames)}).")
            except Exception as e:
                print(f"Error processing {video_file}: {e}")

    features = np.asarray(features, dtype=np.float32)
    labels = np.array(labels, dtype=np.int32)

    print(f"Dataset creation completed: {len(features)} samples collected.")
    return features, labels, video_paths


In [13]:
# Create the dataset
features, labels, video_paths = build_dataset(dataset_path, class_labels, sequence_length, extract_frames)


Processing class: NonViolence (1/2)
Processing class: Violence (2/2)
Dataset creation completed: 2000 samples collected.


In [15]:

import numpy as np

# Assuming you have already extracted 'features', 'labels', and 'video_paths'

# Save the data to your Drive
np.save("/content/drive/MyDrive/features.npy", features)
np.save("/content/drive/MyDrive/labels.npy", labels)
np.save("/content/drive/MyDrive/video_paths.npy", video_paths)

print("Data saved to Google Drive successfully!")

Data saved to Google Drive successfully!


In [16]:
# features, labels, video_paths = np.load("/kaggle/working/features.npy") , np.load("/kaggle/working/labels.npy") ,  np.load("/kaggle/working/video_paths.npy")

# Encoding and Splitting Training-Testing Sets

In [17]:
# Convert labels into one-hot-encoded vectors
one_hot_encoded_labels = to_categorical(labels)

In [18]:
# Split the Data into Train ( 90% ) and Test Set ( 10% ).
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels, test_size = 0.1,
                                                                            shuffle = True, random_state = 42)

In [19]:
print(features_train.shape,labels_train.shape )
print(features_test.shape, labels_test.shape)

(1800, 16, 96, 96, 3) (1800, 2)
(200, 16, 96, 96, 3) (200, 2)


# Importing MobileNet and Fine-Tuning it

In [20]:
from tensorflow.keras.applications import MobileNetV2

# Load the MobileNetV2 model with pre-trained ImageNet weights
# Set include_top to False to exclude the classification head
# Specify input shape as (128, 128, 3)
mobile_net_model = MobileNetV2(weights= "imagenet", include_top=False, input_shape=(frame_height, frame_width, 3))

# Fine-tune the model: Freeze all layers except the last 40
mobile_net_model.trainable = True
for layer in mobile_net_model.layers[:-40]:
    layer.trainable = False

# Optionally display the summary of the model (commented out)
mobile_net_model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "mobilenetv2_1.00_96"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 96, 96, 3)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 48, 48, 32)     │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 48, 48, 32)     │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 48, 48, 32)     │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 48, 48, 32)     │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 48, 48, 32)     │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 48, 48, 32)     │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 48, 48, 16)     │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 48, 48, 16)     │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 48, 48, 96)     │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 48, 48, 96)     │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 48, 48, 96)     │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 49, 49, 96)     │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 24, 24, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise_BN 

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 1,681,536 (6.41 MB)

 Non-trainable params: 576,448 (2.20 MB)

# Building The Model

In [21]:
from tensorflow.keras.regularizers import l2

def build_model():
    model = Sequential()

    # Input layer to match the shape of the features
    model.add(Input(shape=(sequence_length, frame_height, frame_width, 3)))

    # Apply MobileNet in a TimeDistributed manner to process each frame individually
    model.add(TimeDistributed(mobile_net_model))

    # Thêm Batch Normalization để tăng tính ổn định trong quá trình huấn luyện
    model.add(TimeDistributed(BatchNormalization()))

    # Add Dropout layer to reduce overfitting
    model.add(Dropout(0.3))

    # Flatten each frame's output
    model.add(TimeDistributed(Flatten()))

    # Define LSTM layers with L2 Regularization
    lstm_forward = LSTM(units=64, kernel_regularizer=l2(0.01))  # Regularization added here
    lstm_backward = LSTM(units=64, go_backwards=True, kernel_regularizer=l2(0.01))  # Regularization added here

    # Apply Bidirectional LSTM to process the sequence of frames
    model.add(Bidirectional(lstm_forward, backward_layer=lstm_backward))

    # Thêm Batch Normalization sau LSTM để tăng tính ổn định
    model.add(BatchNormalization())

    # Add Dropout to prevent overfitting
    model.add(Dropout(0.3))

    # Add fully connected Dense layers with ReLU activation and L2 Regularization
    model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))  # Regularization added here
    model.add(Dropout(0.3))

    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))  # Regularization added here
    model.add(Dropout(0.3))

    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))  # Regularization added here
    model.add(Dropout(0.3))

    model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))  # Regularization added here
    model.add(Dropout(0.3))

    # Output layer with softmax activation for multi-class classification
    model.add(Dense(len(class_labels), activation='softmax'))

    # Display the summary of the model
    model.summary()

    return model


In [31]:
# Build the model using the defined function
MoBiLSTM_model = build_model()

# Visualize and save the structure of the constructed MobBiLSTM model
from tensorflow.keras.utils import plot_model
plot_model(
    MoBiLSTM_model,
    to_file='MobBiLSTM_model_structure_plot.png',
    show_shapes=True,
    show_layer_names=True
)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed_3 (TimeDistributed) │ (None, 16, 3, 3, 1280)      │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_4 (TimeDistributed) │ (None, 16, 3, 3, 1280)      │           5,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 16, 3, 3, 1280)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_5 (TimeDistributed) │ (None, 16, 11520)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 128)                 │       5,931,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 2)                   │              66 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,271,458 (31.55 MB)

 Trainable params: 7,692,194 (29.34 MB)

 Non-trainable params: 579,264 (2.21 MB)

You must install pydot (`pip install pydot`) for `plot_model` to work.


# Specifying Callbacks and Fitting

In [32]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Tạo EarlyStopping callback để theo dõi val_accuracy và dừng sớm nếu không cải thiện
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=40,
    restore_best_weights=True,
    verbose=1
)

# Tạo ReduceLROnPlateau callback để giảm learning rate nếu val_loss không cải thiện
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.6,
    patience=10,
    min_lr=0.00005,
    verbose=1
)

#Biên dịch mô hình với loss function, optimizer, và metric
MoBiLSTM_model.compile(
    loss='categorical_crossentropy',
    optimizer='sgd',
    metrics=['accuracy']
)




In [34]:
!pip install tensorflow-gpu==2.12.0
# Install a specific version of tensorflow-gpu known to be stable
!pip install --upgrade tensorflow-io #Upgrade the tensorflow-io library

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 26.6 MB/s eta 0:00:00


In [35]:
import tensorflow as tf
print(tf.__version__) # Check Tensorflow Version
print(tf.config.list_physical_devices()) # Verify if the GPU is detected

2.18.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:TPU_SYSTEM:0', device_type='TPU_SYSTEM'), PhysicalDevice(name='/physical_device:TPU:0', device_type='TPU'), PhysicalDevice(name='/physical_device:TPU:1', device_type='TPU'), PhysicalDevice(name='/physical_device:TPU:2', device_type='TPU'), PhysicalDevice(name='/physical_device:TPU:3', device_type='TPU'), PhysicalDevice(name='/physical_device:TPU:4', device_type='TPU'), PhysicalDevice(name='/physical_device:TPU:5', device_type='TPU'), PhysicalDevice(name='/physical_device:TPU:6', device_type='TPU'), PhysicalDevice(name='/physical_device:TPU:7', device_type='TPU')]


In [37]:
MoBiLSTM_model_history = MoBiLSTM_model.fit(
    x=features_train,
    y=labels_train,
    epochs=50,
    batch_size=8,
    shuffle=True,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/50


NotFoundError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.11/dist-packages/tornado/ioloop.

In [ ]:
model_evaluation_history = MoBiLSTM_model.evaluate(features_test, labels_test)

# Model Evaluation

In [ ]:
def plot_metric(model_training_history, metric_name_1, metric_name_2, plot_name):

    metric_value_1 = model_training_history.history[metric_name_1]
    metric_value_2 = model_training_history.history[metric_name_2]

    # Get the Epochs Count
    epochs = range(len(metric_value_1))

    plt.plot(epochs, metric_value_1, 'blue', label = metric_name_1)
    plt.plot(epochs, metric_value_2, 'orange', label = metric_name_2)

    plt.title(str(plot_name))

    plt.legend()

In [ ]:
import matplotlib.pyplot as plt

# Vẽ biểu đồ metric của mô hình
plot_metric(MoBiLSTM_model_history, 'loss', 'val_loss', 'Total Loss vs Total Validation Loss')

# Lưu biểu đồ dưới dạng tệp hình ảnh
plt.savefig('loss_vs_val_loss_plot.png', dpi=300, bbox_inches='tight')

# Hiển thị biểu đồ trên màn hình
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Vẽ biểu đồ accuracy của mô hình
plot_metric(MoBiLSTM_model_history, 'accuracy', 'val_accuracy', 'Total Accuracy vs Validation Accuracy')

# Lưu biểu đồ dưới dạng tệp hình ảnh
plt.savefig('accuracy_vs_val_accuracy_plot.png', dpi=300, bbox_inches='tight')

# Hiển thị biểu đồ trên màn hình
plt.show()


# **Predicting the Test Set**

In [ ]:
labels_predict = MoBiLSTM_model.predict(features_test)

In [ ]:
# Decoding the data to use in Metrics
labels_predict = np.argmax(labels_predict , axis=1)
labels_test_normal = np.argmax(labels_test , axis=1)

In [ ]:
labels_test_normal.shape , labels_predict.shape

# Accuracy Score

In [ ]:
from sklearn.metrics import accuracy_score
AccScore = accuracy_score(labels_predict, labels_test_normal)
print('Accuracy Score is : ', AccScore)

# Confusion Matrix

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

# Compute the confusion matrix
cm = confusion_matrix(labels_test_normal, labels_predict)

# Create a subplot for the heatmap
fig, ax = plt.subplots()

# Plot the heatmap
sns.heatmap(cm, annot=True, fmt='d', ax=ax, cmap='coolwarm', linewidths=0.5)

# Set the labels and title
ax.set_xlabel('Predicted Labels', fontsize=12)
ax.set_ylabel('True Labels', fontsize=12)
ax.set_title('Confusion Matrix', fontsize=14)

# Set tick labels
ax.set_xticklabels(['True', 'False'], fontsize=10)
ax.set_yticklabels(['NonViolence', 'Violence'], fontsize=10, rotation=0)

# Adjust layout for better spacing
plt.tight_layout()

# Show the plot
plt.show()


Classification Report

In [ ]:
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(labels_test_normal, labels_predict)

# Print the classification report with a custom message
print("Detailed Classification Report:\n")
print(report)


# Prediction Frame By Frame

In [ ]:
def predict_frames(video_file_path, output_file_path, sequence_length):

    # Read from the video file.
    video_reader = cv2.VideoCapture(video_file_path)

    # Get the width and height of the video.
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # VideoWriter to store the output video in the disk.
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('m', 'p', '4', 'v'),
                                    video_reader.get(cv2.CAP_PROP_FPS), (original_video_width, original_video_height))

    # Declare a queue to store video frames.
    frames_queue = deque(maxlen = sequence_length)

    # Store the predicted class in the video.
    predicted_class_name = ''

    # Iterate until the video is accessed successfully.
    while video_reader.isOpened():

        ok, frame = video_reader.read()

        if not ok:
            break

        # Resize the Frame to fixed Dimensions.
        resized_frame = cv2.resize(frame, (frame_height, frame_width))

        # Normalize the resized frame
        normalized_frame = resized_frame / 255

        # Appending the pre-processed frame into the frames list.
        frames_queue.append(normalized_frame)

        # We Need at Least number of SEQUENCE_LENGTH Frames to perform a prediction.
        # Check if the number of frames in the queue are equal to the fixed sequence length.
        if len(frames_queue) == sequence_length:

            # Pass the normalized frames to the model and get the predicted probabilities.
            predicted_labels_probabilities = MoBiLSTM_model.predict(np.expand_dims(frames_queue, axis = 0))[0]

            # Get the index of class with highest probability.
            predicted_label = np.argmax(predicted_labels_probabilities)

            # Get the class name using the retrieved index.
            predicted_class_name = class_labels[predicted_label]

        # Write predicted class name on top of the frame.
        if predicted_class_name == "Violence":
            cv2.putText(frame, predicted_class_name, (5, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 12)
        else:
            cv2.putText(frame, predicted_class_name, (5, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 255, 0), 12)

        # Write The frame into the disk using the VideoWriter
        video_writer.write(frame)

    video_reader.release()
    video_writer.release()

In [ ]:
plt.style.use("default")

# To show Random Frames from the saved output predicted video (output predicted video doesn't show on the notebook but can be downloaded)
def show_pred_frames(pred_video_path):

    plt.figure(figsize=(20,15))

    video_reader = cv2.VideoCapture(pred_video_path)

    # Get the number of frames in the video.
    frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Get Random Frames from the video then Sort it
    random_range = sorted(random.sample(range (sequence_length , frames_count ), 12))

    for counter, random_index in enumerate(random_range, 1):

        plt.subplot(5, 4, counter)

        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, random_index)

        ok, frame = video_reader.read()

        if not ok:
          break

        frame = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)

        plt.imshow(frame);ax.figure.set_size_inches(20,20);plt.tight_layout()

    video_reader.release()

In [ ]:
# Construct the output video path.
test_videos_directory = 'test_videos'
os.makedirs(test_videos_directory, exist_ok = True)

output_video_file_path = f'{test_videos_directory}/Output-Test-Video.mp4'

In [ ]:
# Specifying video to be predicted
input_video_file_path = "../input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/V_378.mp4"

# Perform Prediction on the Test Video.
predict_frames(input_video_file_path, output_video_file_path, sequence_length)

# Show random frames from the output video
show_pred_frames(output_video_file_path)

In [ ]:
# Specifying video to be predicted
input_video_file_path = "../input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/V_118.mp4"

# Perform Prediction on the Test Video.
predict_frames(input_video_file_path, output_video_file_path, sequence_length)

# Show random frames from the output video
show_pred_frames(output_video_file_path)

In [ ]:
# Specifying video to be predicted
input_video_file_path = "../input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/V_200.mp4"

# Perform Prediction on the Test Video.
predict_frames(input_video_file_path, output_video_file_path, sequence_length)

# Show random frames from the output video
show_pred_frames(output_video_file_path)

In [ ]:
# Specifying video to be predicted
input_video_file_path = "../input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/V_100.mp4"

# Perform Prediction on the Test Video.
predict_frames(input_video_file_path, output_video_file_path, sequence_length)

# Show random frames from the output video
show_pred_frames(output_video_file_path)

In [ ]:
# Play the actual video
Play_Video(input_video_file_path)

In [ ]:
# Specifying video to be predicted
input_video_file_path = "../input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/NV_1.mp4"

# Perform Prediction on the Test Video.
predict_frames(input_video_file_path, output_video_file_path, sequence_length)

# Show random frames from the output video
show_pred_frames(output_video_file_path)

In [ ]:
# Specifying video to be predicted
input_video_file_path = "../input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/NV_100.mp4"

# Perform Prediction on the Test Video.
predict_frames(input_video_file_path, output_video_file_path, sequence_length)

# Show random frames from the output video
show_pred_frames(output_video_file_path)

In [ ]:
# Specifying video to be predicted
input_video_file_path = "../input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/NV_244.mp4"

# Perform Prediction on the Test Video.
predict_frames(input_video_file_path, output_video_file_path, sequence_length)

# Show random frames from the output video
show_pred_frames(output_video_file_path)

In [ ]:
# Play the actual video
Play_Video(input_video_file_path)